In [ ]:

def getAllUsersInMonth(inMonth, consumer, delivery, ordercart, restaurant):
    # dict to store users keyed by month
    result = set()
    for order in ordercart:
        user = order.creator_id
        month = getMonth(delivery.get(order.delivery_id).actual_delivery_time)
        if month == inMonth:
            result.add(user)
    return result

def getMonhlyNewCohortsByMonth(consumer, delivery, ordercart, restaurant):
    # dict to store users keyed by month
    cohorts_by_joined_month = {}
    for order in ordercart:
        if order.is_first_ordercart:
            user = order.creator_id
            month = getMonth(delivery.get(order.delivery_id).actual_delivery_time)
            if month in cohorts_by_joined_month:
                cohorts_by_joined_month[month].add(user)
            else:
                cohorts_by_joined_month[month] = set([user])
                
    return cohorts_by_joined_month
                
def getMonhlyCohortsByMonthAndMarket(consumer, delivery, ordercart, restaurant):
    # dict to store result keyed by month, and further by market
    cohorts_by_joined_month_and_market = {}
    for order in ordercart:
        if order.is_first_ordercart:
            user = order.creator_id
            delivery_row = delivery.get(order.delivery_id)
            month = getMonth(delivery_row.actual_delivery_time)
            restaurant_row = restaurant.get(delivery_row.restaurant_id)
            market = restaurant_row.market_id
            if month in cohorts_by_joined_month_and_market:
                market_data = cohorts_by_joined_month_and_market[month]
            else:
                market_data = {}
                cohorts_by_joined_month_and_market[month] = { market: market_data }

            if market in market_data:
                market_data[market].add(user)
            else:
                market_data[market] = set([user])
                
    return cohorts_by_joined_month_and_market

        
def getRetentionByMonth(consumer, delivery, ordercart, restaurant, xmonth=1):
    newCohortsByMonth = getMonhlyNewCohortsByMonth()
    retentionByMonth = {}
    for month, users in newCohortsByMonth.items():
        nextMonth = getMonthAfterXMonths(month, xmonth)
        nextMonthUsers = getAllUsersInMonth(nextMonth)
        retainedUsers = nextMonthUsers.intersect(users)
        
        # we keep user and retained user count for each month for later analysis
        retentionByMonth[month] = ((100.0 * len(retainedUsers)) / len(users), len(retainedUsers), len(users))
    return retentionByMonth
        
def getWeightedAvgRetention(monthDiff):
    total_retained, total_users = 0, 0
    retentionByMonth = getRetentionByMonth(consumer, delivery, ordercart, restaurant, xmonth=monthDiff)
    for month, data for retentionByMonth.items():
        percent, rusers, users = data
        total_retained = rusers
        total_users = users
        
    return (100.0 * total_retained / total_users)

def getRetentionByMonthAndMarket(consumer, delivery, ordercart, restaurant, xmonth=1):
    newCohortsByMonthAndMarket = getMonhlyCohortsByMonthAndMarket()
    retentionByMonthAndMarket = {}
    for month, usersByMarket in newCohortsByMonthAndMarket.items():
        retentionByMonthAndMarket[month] = {}
        nextMonth = getMonthAfterXMonths(month, xmonth)
        nextMonthUsers = getAllUsersInMonth(nextMonth)
        for market, users in usersByMarket.items():
            retainedUsers = nextMonthUsers.intersect(users)
            # we keep user and retained user count for each month for later analysis
            retentionByMonthAndMarket[month][market] = ((100.0 * len(retainedUsers)) / len(users), len(retainedUsers), len(users))
    return retentionByMonthAndMarket
        
def getWeightedAvgRetentionByMarket(monthDiff):
    retentionByMonthAndMarket = getRetentionByMonthAndMarket(consumer, delivery, ordercart, restaurant, xmonth=monthDiff)
    markets = getAllMarkets() # not implemented yet

    weightedRetentionByMarket = {}
    for market in markets:
        total_retained, total_users = 0, 0
        for month, data for retentionByMonth.items():
            data2 = data[market]
            percent, rusers, users = data
            total_retained = rusers
            total_users = users
        marketRetention = (100.0 * total_retained / total_users)
        weightedRetentionByMarket[market] = marketRetention
        
    return weightedRetentionByMarket

        
    
    
    
